In [5]:
# coding: utf-8

In [1]:
import os
import sys
from pathlib import Path
import pandas as pd
import sqlite3
import sqlalchemy as sa
from sqlalchemy import create_engine
from sqlalchemy.sql import select
from datetime import datetime

In [2]:
sys.path.append(r"../src/collectcube")
from db import *

In [3]:
local_db_path = '../data/example/landcover.db'

### Check that pixel table already exists

In [4]:
pixdf = check_table(local_db_path,'pixels')

          PID  Center     cent_X     cent_Y   cent_lat  cent_long  ransamp  \
85  0000009_4       0  3022810.0 -3026470.0 -23.906908 -57.100368        1   
86  0000009_5       0  3022830.0 -3026470.0 -23.906908 -57.100150        1   
87  0000009_6       0  3022810.0 -3026480.0 -23.906990 -57.100358        1   
88  0000009_7       0  3022820.0 -3026480.0 -23.906990 -57.100250        1   
89  0000009_8       0  3022830.0 -3026480.0 -23.906990 -57.100141        1   

    checked  PID0  PID1 sampgroup  
85        0     9     4  example2  
86        0     9     5  example2  
87        0     9     6  example2  
88        0     9     7  example2  
89        0     9     8  example2  


## Make sql lc table from existing lut.csv

In [5]:
lc_lut = Path("../data/Class_LUT.csv")
make_LC_table_from_lut(lc_lut, local_db_path, treat_existing='fail')

In [6]:
make_simp_LC_table(lc_lut, local_db_path, [0,30,35,40,80,98], treat_existing='fail')

### check if it worked:

In [8]:
lcdf = check_table(local_db_path,'LC')

    LC_UNQ           USE_NAME LC5_name  LC5  LC25
39      68  Trees-Forest_palm    Trees   70    68
40      70      Trees-Wetland    Trees   70    70
41      80       Trees-Forest    Trees   70    80
42      98           N_NoCrop  unknown   99    98
43      99            unknown  unknown   99    99


### Add other tables to database using sqlalchemy to control foreign keys

In [8]:
def make_db(db_path):
    sql_db_path = 'sqlite:///'+ db_path
    engine = create_engine(sql_db_path, echo=False)
    make_main_support_tables(engine)
    populate_LC5_table(engine)
    
make_db(local_db_path)

added LC5 and PixVar tables to db


### Check that tables are made and populated

In [12]:
engine = create_engine('sqlite:///'+ local_db_path)
with engine.connect() as conn:
    lc5_table = Table('LC5', MetaData(), autoload_with=engine)
    rp = conn.execute(lc5_table.select())
results = rp.fetchall() 
print(results)

[(0, ' ------'), (10, 'NoVeg'), (20, 'LowVeg'), (40, 'MedVeg'), (50, 'HighVeg'), (70, 'Trees'), (99, 'Unknown')]


In [13]:
with engine.connect() as con:
    df = pd.read_sql_table('LC5', con)
df.head()

,LC5id,LC5type
0,0,------
1,10,NoVeg
2,20,LowVeg
3,40,MedVeg
4,50,HighVeg


In [14]:
with engine.connect() as con:
    df = pd.read_sql_table('PixelVerification', con)
df.head()

,recID,PID,PID0,PID1,imgDate,LC5,LC,HOMONBHD9,ForestProx,WaterProx,...,DEAD,MEDVEG,TREEPLANT0,HIGHVEG,TREEPLANT,FOREST,Age,Stability,State,Notes


In [15]:
## first record:
testDate = datetime(2000,1,1)
with engine.connect() as conn:
    pixvar = Table('PixelVerification', MetaData(), autoload_with=engine)
    query = sa.insert(pixvar).values(PID='0_0', PID0=0, PID1=0, imgDate=testDate,LC5=0,LC=0,HOMONBHD9=8) 
    go = conn.execute(query)
    conn.commit()
print(go.inserted_primary_key)

(1,)
